[Advent of Code - Day 07](https://adventofcode.com/2019/day/7)

# Import input

In [1]:
import sys
from itertools import permutations

sys.path.insert(0, "../")
from utils import aoc_input as inp

in_ = inp.download_input(year="2019", day="07")[0]

In [2]:
in_[:5]

'3,8,1'

# Solulu

In [3]:
def solve(input_values, intcode: list, pointer: int = 0, part: int = 1):
    output_index = 0
    input_count = 0 if pointer == 0 else 1
    used_input = False

    while True:
        opcode, modes, total_values = decode(intcode[pointer])

        if opcode == 99:
            break
            
        elif opcode == 4:
            output_index = intcode[pointer + 1]

        elif opcode == 3:
            if used_input:
                break
            input_index = intcode[pointer + 1]
            intcode[input_index] = input_values[input_count]
            if input_count == 0:
                input_count += 1
            elif input_count == 1 and part == 2:
                used_input = True
                
        elif opcode in [1, 2, 7, 8]:
            nums = []
            for i, mode in enumerate(modes[:-1], start=1):
                num = parameter_mode(mode, pointer, i, intcode)
                nums.append(num)
            if opcode == 1:
                value = nums[0] + nums[1]
            elif opcode == 2:
                value = nums[0] * nums[1]
            elif opcode == 7:
                value = nums[0] < nums[1]
            elif opcode == 8:
                value = nums[0] == nums[1]
            value_index = intcode[pointer + 3]
            intcode[value_index] = value

        if opcode == 5 and parameter_mode(modes[0], pointer, 1, intcode) != 0:
            pointer = parameter_mode(modes[1], pointer, 2, intcode)
        elif opcode == 6 and parameter_mode(modes[0], pointer, 1, intcode) == 0:
            pointer = parameter_mode(modes[1], pointer, 2, intcode)
        else:
            pointer += total_values

    output_value = intcode[output_index]

    return output_value, intcode, pointer, opcode


def parameter_mode(mode, pointer, param, intcode) -> int:
    if mode == 0:
        num_index = intcode[pointer + param]
    elif mode == 1:
        num_index = pointer + param
    num = intcode[num_index]

    return num


def decode(instruction):
    opcode = instruction % 100
    modes = list()
    total_values = 1
    if opcode in (3, 4):
        total_values += 1
    else:
        mode1 = (instruction // 100) % 10
        mode2 = instruction // 1000
        modes.extend([mode1, mode2])
        if opcode not in (5, 6):
            mode3 = instruction // 10000
            modes.append(mode3)

    total_values += len(modes)

    return opcode, modes, total_values


def _reset_memory(inp=in_):
    return list(map(int, in_.split(",")))

## Part 1

In [4]:
no_amplifiers = 5
combos = permutations([i for i in range(no_amplifiers)])
max_output_signal = 0

for combo in combos:
    input_signal = 0

    for phase in combo:
        intcode = _reset_memory()
        output_signal, _, _, _ = solve(
            input_values=[phase, input_signal], intcode=intcode
        )
        input_signal = output_signal

    max_output_signal = max(output_signal, max_output_signal)

max_output_signal

272368

## Part 2

In [5]:
no_amplifiers = 5
combos = permutations([i for i in range(no_amplifiers, no_amplifiers * 2)])
max_output_signal = 0

for combo in combos:
    input_signal = 0
    current = 0
    amplifiers = {
        0: {"phase": combo[0], "memory": None, "pointer": 0, "input": input_signal},
        1: {"phase": combo[1], "memory": None, "pointer": 0, "input": None},
        2: {"phase": combo[2], "memory": None, "pointer": 0, "input": None},
        3: {"phase": combo[3], "memory": None, "pointer": 0, "input": None},
        4: {"phase": combo[4], "memory": None, "pointer": 0, "input": None},
    }

    while True:
        if not amplifiers[current]["memory"]:
            amplifiers[current]["memory"] = _reset_memory()

        output_signal, intcode, pointer, opcode = solve(
            input_values=[amplifiers[current]["phase"], amplifiers[current]["input"]],
            intcode=amplifiers[current]["memory"],
            pointer=amplifiers[current]["pointer"],
            part=2,
        )

        if opcode == 99 and current == 4:
            max_output_signal = max(output_signal, max_output_signal)
            break

        amplifiers[current]["memory"] = intcode
        amplifiers[current]["pointer"] = pointer
        current = (current + 1) % no_amplifiers
        amplifiers[current]["input"] = output_signal

max_output_signal

19741286